In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now()}')

Päivitetty 2022-10-26 19:03:22.681364


# Ristiintaulukoinnit Exceliin

Datan tilalla voi käyttää mitä tahansa oikeaoppisesti tallennettua dataa.

In [2]:
import pandas as pd

# Excelin ohjaukseen
import xlwings as xw

# Jos ainutkertaisia arvoja enemmän kuin RAJA, niin ei tulkita kategoriseksi
RAJA = 10

# Avaan datan
df = pd.read_excel('http://taanila.fi/data1.xlsx')

In [3]:
# Lista kategorisista muuttujista
kategoriset = []
for var in df:
    if len(df[var].unique()) <= RAJA:
        kategoriset.append(var)

if kategoriset:
    # Excel-työkirja (workbook)
    app = xw.App(visible=False)
    wb = xw.books.active
    
    # Taulukkovälilehden nimeäminen
    ws1 = wb.sheets.active
    ws1.name = 'ristiintaulukoinnit'
        
    # Pidän kirjaa rivinumerosta rivi-muuttujan avulla
    rivi = 4

# for-silmukoilla käyn läpi kaikki muuttujaparit
    for var1 in kategoriset:
        for var2 in kategoriset:
            if var1 != var2:
                df1 = pd.crosstab(df[var1], df[var2], margins=True)
                df2 = pd.crosstab(df[var1], df[var2], normalize='columns')
                df2.loc['n'] = df1.loc['All']
                ws1.range(rivi, 2).value = var2
                ws1.range(rivi, df1.shape[1] + 4).value = var2
                ws1.range(rivi+1, 1).value = df1
                ws1.range(rivi+1, df1.shape[1]+1).value = 'yhteensä'
                ws1.range(rivi+df1.shape[0]+1, 1).value = 'yhteensä'
                ws1.range((rivi+1, df1.shape[1]+3)).value = df2
                ws1.range((rivi+2, df1.shape[1]+4),(rivi+df2.shape[0], 
                df1.shape[1]+3+df2.shape[1])).number_format = '0,0 %'
                rivi = rivi + df2.shape[0] + 3
    ws1.range(1, 1).value = 'Ristiintaulukoinnit kategorisille muuttujille'
    ws1.range(2, 1).value = \
        'Prosentit ovat prosentteja sarakkeen kokonaismäärästä (n)'
    ws1.range((1, 1), (2, 1)).font.bold = True
    
# Näytetään Excel
app.visible = True

Lisätietoa Pythonin käytöstä data-analytiikassa https://tilastoapu.wordpress.com/python/